|Pretrain|Finetune|Inference|
|---|---|---|
|---|[V2: original](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch?scriptVersionId=67432494)|[V2: 0.67](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch-infer?scriptVersionId=67439784)|
|---|[V3: tpu](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch?scriptVersionId=68152775)|[V3: 0.473](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch-infer?scriptVersionId=68153264)|
|---|[V2: len 248](https://www.kaggle.com/azamat25/fork-of-pre-trained-roberta-solution-in-pytorch?scriptVersionId=68521516)|[V4: 0.470](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch-infer?scriptVersionId=68583646)|
|---|[V4: original + modified head ](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch/notebook?scriptVersionId=68737645)|[V5: 0.472]()|
|---|[V5: original + modified head , dynamic padding, lr optimization, grad accumulation](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch?scriptVersionId=68742757)|[V6: 0.473](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch-infer?scriptVersionId=68747885)|
|---|[V6:CLRP-Finetune by algernone + max_len = 248](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch?scriptVersionId=68753898)|[V7: 0.474]()|
|---|[V7: Andrey's original + algernone's head]()|[V8: ]()|
|---|[V1: epochs 3, Second To Last CLS, Last Hidden State's logits, Weighted Pooling Logits, Concat Last 4 Layers](https://www.kaggle.com/azamat25/clrp-roberta-base-finetune?scriptVersionId=69528989)|[V21: 0.464](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch-infer?scriptVersionId=69530140)|
|---|[V2: epochs 5, Second To Last CLS, Mean Pooling, Last Hidden State's logits, Weighted Pooling Logits, Concat Last 4 Layers ](https://www.kaggle.com/azamat25/clrp-roberta-base-finetune?scriptVersionId=69528989)|[V22: 0.472](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch-infer?scriptVersionId=69530368)|
|---|[V4: epochs 4, Second To Last CLS, Last Hidden State's logits, Weighted Pooling Logits, Concat Last 4 Layers, Sum Last 4 Layers](https://www.kaggle.com/azamat25/clrp-roberta-base-finetune/output?scriptVersionId=69546342)|[V23: ](https://www.kaggle.com/azamat25/pre-trained-roberta-solution-in-pytorch-infer?scriptVersionId=69530368)|
|---|[V6: epochs 4, max_len256, Second To Last CLS, Last Hidden State's logits, Weighted Pooling Logits, Concat Last 4 Layers]()|[V24: 0.467]()|

In [1]:
import os
from pathlib import Path
roberta_base_path = Path('../input/final-clrp-roberta-base-finetuned')
roberta_base_dir = Path(roberta_base_path / 'scripts')

In [3]:
os.chdir(roberta_base_dir)
exec(Path("imports.py").read_text())
exec(Path("config.py").read_text())
exec(Path("dataset.py").read_text())
exec(Path("model.py").read_text())
os.chdir('/kaggle/working')

In [4]:
def set_random_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
set_random_seed(seed=42)

In [10]:
test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")
tokenizer = AutoTokenizer.from_pretrained(Config.roberta_model)
base_folder_path = Path(roberta_base_path / 'models')
base_preds = []
n_models = 5

# Inference

In [11]:
SEED = 1000

for model_num in range(n_models):
    set_random_seed(SEED + model_num)
    print(f'Inference#{model_num+1}')
    test_ds = LitDataset(data=test_df, tokenizer=tokenizer, max_len=Config.max_len, is_test=True)
    test_sampler = SequentialSampler(test_ds)
    test_dataloader = DataLoader(test_ds, sampler = test_sampler, batch_size=Config.batch_size)
    model = LitModel(Config.roberta_model)
    model = torch.load(base_folder_path / f'LitModel_{model_num}.pt').to(Config.device)

    roberta_base_preds = []
    model.eval()

    for step,batch in enumerate(test_dataloader):
        sent_id, mask = batch['input_ids'].to(Config.device), batch['attention_mask'].to(Config.device)
        with torch.no_grad():
            preds = model(sent_id, mask)
            roberta_base_preds += preds.flatten().cpu().tolist()
    
    base_preds.append(roberta_base_preds)

Inference#1
Inference#2
Inference#3
Inference#4
Inference#5


In [12]:
base_preds = np.array(base_preds)
print(base_preds.shape)
print(base_preds)
roberta_base_preds = base_preds.mean(axis=0)
print(roberta_base_preds.shape)

(5, 7)
[[-0.50063735 -0.68645883 -0.29343396 -2.57816935 -1.73586512 -1.13054061
  -0.05757669]
 [-0.56450933 -0.63375056 -0.17978388 -2.38634682 -1.84923458 -1.70842803
   0.0764083 ]
 [-0.36793834 -0.6650098  -0.53269571 -2.48349166 -1.54117477 -1.54048097
  -0.12390154]
 [-0.42983121 -0.57754916 -0.38004464 -2.42542839 -1.90584028 -1.12281227
   0.00817369]
 [-0.51568878 -0.42094815 -0.47697312 -2.91067004 -1.9208709  -1.51525152
   0.21890225]]
(7,)


In [13]:
submission_df.target = roberta_base_preds 
print(submission_df)
submission_df.to_csv("submission.csv", index=False)

          id    target
0  c0f722661 -0.475721
1  f0953f0a5 -0.596743
2  0df072751 -0.372586
3  04caf4e0c -2.556821
4  0e63f8bea -1.790597
5  12537fe78 -1.403503
6  965e592c0  0.024401
